In [ ]:
%reload_ext autoreload
%autoreload 2

## Building our own models for CCS prediction

In [ ]:
from peptdeep.model.featurize import (
    get_batch_aa_indices, 
    get_batch_mod_feature
)

from peptdeep.settings import model_const

import peptdeep.model.model_interface as model_base

mod_feature_size = len(model_const['mod_elements'])

import torch
import pandas as pd

In [ ]:
class CCS_LSTM_Module(torch.nn.Module):
    def __init__(self,
        dropout=0.1,
    ):
        super().__init__()
        
        self.dropout = torch.nn.Dropout(dropout)
        
        hidden = 256

        self.ccs_encoder = (
            model_base.Encoder_AA_Mod_Charge_CNN_LSTM_AttnSum(
                hidden
            )
        )

        self.ccs_decoder = model_base.Decoder_Linear(
            hidden+1, 1
        )

    def forward(self, 
        aa_indices, 
        mod_x,
        charges,
    ):
        x = self.ccs_encoder(aa_indices, mod_x, charges)
        x = self.dropout(x)
        x = torch.cat((x, charges),1)
        return self.ccs_decoder(x).squeeze(1)

In [ ]:
class CCS_Model(model_base.PeptideModelInterfaceBase):
    def __init__(self, 
        model_class:torch.nn.Module=CCS_LSTM_Module,
        dropout=0.1,
    ):
        super().__init__()
        self.build(
            model_class,
            dropout=dropout,
        )
        self.loss_func = torch.nn.L1Loss()

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'ccs_pred'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
    ):
        aa_indices = torch.LongTensor(
            get_batch_aa_indices(
                batch_df['sequence'].values.astype('U')
            )
        )

        mod_x = torch.Tensor(
            get_batch_mod_feature(
                batch_df
            )
        )

        charges = torch.Tensor(
            batch_df['charge'].values
        ).unsqueeze(1)*0.1

        return aa_indices, mod_x, charges

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame,
    ) -> torch.Tensor:
        return torch.Tensor(batch_df['ccs'].values)

## Testing the CCS model

### Prepare training data

In [ ]:
from peptdeep.model.rt import irt_pep
# virtual ccs values for training
irt_pep['ccs'] = pd.RangeIndex(0, len(irt_pep)).values.astype(float)
irt_pep['charge'] = 2
irt_pep

,sequence,pep_name,irt,mods,mod_sites,nAA,ccs,charge
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.0,2
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1.0,2
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,2.0,2
3,YILAGVENSK,RT-pep d,19.79,,,10,3.0,2
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,4.0,2
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,5.0,2
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,6.0,2
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,7.0,2
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,8.0,2
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,9.0,2


In [ ]:
ccs_model = CCS_Model()

### Test the untrained model

### Test if training works

In [ ]:
ccs_model.train(irt_pep, epoch=100, verbose=False)

### Test if the model fits the virtual ccs values

In [ ]:
ccs_model.predict(irt_pep)

,sequence,pep_name,irt,mods,mod_sites,nAA,ccs,charge,ccs_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.0,2,0.001722
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1.0,2,1.137457
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,2.0,2,2.255439
3,YILAGVENSK,RT-pep d,19.79,,,10,3.0,2,3.415788
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,4.0,2,4.315064
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,5.0,2,5.311618
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,6.0,2,6.441224
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,7.0,2,7.204535
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,8.0,2,8.018152
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,9.0,2,8.974627
